In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import numpy.ma as ma
import struct
import os
import xarray as xr
import glob
import datetime
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
# ESIO Imports
import esio

In [ ]:
# General plotting settings
sns.set_style('whitegrid')
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})

In [ ]:
# Load in Obs
data_dir = r'/home/disk/sipn/nicway/data/obs/'
temp_dir = r'/home/disk/sipn/nicway/data/model/temp' # Used for temp storing netcdf files
fig_dir = r'/home/disk/sipn/nicway/python/ESIO/figures'
da_51 = xr.open_mfdataset(os.path.join(data_dir, 'NSIDC_0051/sipn_nc/NSIDC_0051.nc'))
# da_79 = xr.open_mfdataset(os.path.join(data_dir, 'NSIDC_0079/sipn_nc/NSIDC_0079.nc'))
# da_81 = xr.open_mfdataset(os.path.join(data_dir, 'NSIDC_0081/sipn_nc/NSIDC_0081.nc'))

In [ ]:
# Load in Model
nmme_refor_dir = r'/home/disk/sipn/nicway/data/model/nmme/reforecast/sipn_nc/GFDL_FLORB01_*200*.nc'
ds_gfdl_flor_in = xr.open_mfdataset(nmme_refor_dir)
ds_gfdl_flor_in

In [ ]:
# Mask out to common extent (both observations and model have non-null values)
(da_51, ds_gfdl_flor) = esio.mask_common_extent(da_51, ds_gfdl_flor_in)

In [ ]:
# Aggregate over domain
da_51_avg = da_51.sum(dim='x').sum(dim='y')*(25*25)/(10**6)
# da_79_avg = da_79.sum(dim='x').sum(dim='y')*(25*25)/(10**6)
# da_81_avg = da_81.sum(dim='x').sum(dim='y')*(25*25)/(10**6)
ds_gfdl_flor_avg = ds_gfdl_flor.sic.sum(dim='x').sum(dim='y')*(25*25)/(10**6)

In [ ]:
# Aggreagate Obs to Model temporal time stamp
# ex: gfdl data is monthly means, time stamp at beinging of period
da_51_avg_mon = da_51_avg.resample(time='MS').mean()
# da_79_avg_mon = da_79_avg.resample(time='MS').mean()
# da_81_avg_mon = da_81_avg.resample(time='MS').mean()

In [ ]:
# Trim to common time periods
(ds_obs_out, ds_mod_out) = esio.trim_common_times(da_51_avg_mon, ds_gfdl_flor_avg)

In [ ]:
# # Temp dump to netcdf then load
# os.chdir( temp_dir )
# init_time, datasets = zip(*ds_mod_out.to_dataset(name='sic').groupby('init_time'))
# paths = ['GFDL_init_time_%s.nc' % i for i in init_time]
# xr.save_mfdataset(datasets, paths)
# # ds_mod_out.save_mfdataset(os.path.join(temp_dir, 'temp.nc'))

In [ ]:
# ds_mod_out = xr.open_mfdataset(os.path.join(temp_dir, 'GFDL_init_time*.nc'), concat_dim='init_time')

In [ ]:
ds_mod_out

In [ ]:
ds_obs_out.sic

In [ ]:
# Slow way... loop over each init_time and forecast time, calcuate metric

In [ ]:
# Format obs like model
da_51_avg_mon_X = esio.format_obs_like_model(ds_mod_out, ds_obs_out)

In [ ]:
 # Get observational mean and sigma
(mu, sigma) = esio.clim_mu_sigma(da_51_avg_mon, method='MK')

In [ ]:
c_nrmse = esio.NRMSE(ds_gfdl_flor_avg, da_51_avg_mon_X, sigma)

In [ ]:
#NRMSE is following the same pattern per months as Hawkins et al. 2016.
f, ax1 = plt.subplots(1,1)
f.set_size_inches(20, 10)
c_nrmse.plot(ax=ax1)

In [ ]:
# Plot pan-Arctic sea ice extent
f, ax1 = plt.subplots(1,1)
f.set_size_inches(20, 10)
# Observations
ds_obs_out.plot(ax=ax1, label='NSIDC Bootstrap (Comiso et al. 2017)')
plt.ylabel('Millions of square km')
# Models
mmm = ds_mod_out.mean(dim='ensemble').isel(fore_time_i=0)
mmm.plot(ax=ax1, label='GFDL Ensemble Mean')
# esio.plot_model_ensm(ds_mod_out, axin=ax1, labelin='GFDL FLORB01 Model Ensembles')
plt.legend(bbox_to_anchor=(1.03, 1.05))
# plt.xlim([ds_gfdl_flor_avg.fore_time.min().values, ds_gfdl_flor_avg.fore_time.max().values])
f.savefig(os.path.join(fig_dir,'panArcticSIC.png'),bbox_inches='tight',dpi=200)

In [ ]:
# Plot pan-Arctic sea ice extent
f, ax1 = plt.subplots(1,1)
f.set_size_inches(20, 10)
# Observations
da_51_avg.plot(ax=ax1, label='NSIDC Historical (Cavalieri et al. 1996)')
da_79_avg.plot(ax=ax1, label='NSIDC Bootstrap (Comiso et al. 2017)')
da_81_avg.plot(ax=ax1, label='NSIDC Near-Real-Time (Maslanik et al. 1999)')
plt.ylabel('Millions of square km')
# Models
# esio.plot_model_ensm(ds_gfdl_flor_avg, axin=ax1, labelin='GFDL FLORB01 Model Ensembles')
plt.legend(bbox_to_anchor=(1.03, 1.05))
f.savefig(os.path.join(fig_dir,'panArcticSIC.png'),bbox_inches='tight',dpi=200)

In [ ]:
# # Plot difference between NSIDC Historical and NRT data set
# plt.figure(figsize=(10,5))
# (da_51_avg-da_81_avg).plot(label='NSIDC Historical - Near Real Time')
# plt.ylabel('Millions of square km')
# plt.legend(bbox_to_anchor=(1.03, 1.05))

In [ ]:
# # Plot pan-Arctic sea ice extent RATE (daily)
# f=plt.figure(figsize=(20,10))
# da_51_avg.diff(dim='time').plot(label='NSIDC Historical (Cavalieri et al. 1996)')
# da_79_avg.diff(dim='time').plot(label='NSIDC Bootstrap (Comiso et al. 2017)')
# da_81_avg.diff(dim='time').plot(label='NSIDC Near-Real-Time (Maslanik et al. 1999)')
# plt.ylabel('Millions of square km\n per day')
# plt.legend(bbox_to_anchor=(1.03, 1.05))
# plt.ylim([-0.4, 0.4])
# f.savefig(os.path.join(fig_dir,'panArcticSIC.png'),bbox_inches='tight',dpi=200)

In [ ]:
# # Plot difference between NSIDC Historical and NRT data set RATES
# plt.figure(figsize=(10,5))
# (da_51_avg.diff(dim='time')-da_81_avg.diff(dim='time')).plot(label='NSIDC Historical - Near Real Time')
# plt.ylabel('Millions of square km')
# plt.legend(bbox_to_anchor=(1.03, 1.05))
# plt.ylim([-0.1, 0.1])

In [ ]:
# Set up plotting info
cmap_sic = matplotlib.colors.ListedColormap(sns.color_palette("Blues", 10))
cmap_sic.set_bad(color = 'lightgrey')
cmap_dif = matplotlib.colors.ListedColormap(sns.color_palette("RdBu", 10))
cmap_dif.set_bad(color = 'lightgrey')

In [ ]:
# Plot coverage on a certain date
cdate = '2013-01-16'
c_sic = ds_gfdl_flor.isel(fore_time_i=0)
c_sic = c_sic.mean(dim='ensemble').sel(init_time=cdate, method='nearest').sic.squeeze()

In [ ]:
# Plot Obs and model SIC for date
f = plt.figure(figsize=(30,30));

# Model
ax1 = plt.subplot(3, 1, 1, projection=ccrs.NorthPolarStereo(central_longitude=0))
c_sic.plot.pcolormesh(ax=ax1, x='lon', y='lat', 
                                     transform=ccrs.PlateCarree(),
                                     cmap=cmap_sic,
                      vmin=0, vmax=1)
ax1.set_title('GFDL')
# Add coastlines and meridians/parallels 
ax1.coastlines(linewidth=0.75, color='black', resolution='50m')
ax1.gridlines(crs=ccrs.PlateCarree(), linestyle='-')
ax1.set_extent([-180, 180, 50, 90], crs=ccrs.PlateCarree())



# Obs NSIDC 0051
ax2 = plt.subplot(3, 1, 2, projection=ccrs.NorthPolarStereo(central_longitude=0))
da_51.sel(time=cdate).plot.pcolormesh(ax=ax2, x='lon', y='lat', 
                                     transform=ccrs.PlateCarree(),
                                     cmap=cmap_sic,
                      vmin=0, vmax=1)
ax2.set_title('NSIDC 0051')
# Add coastlines and meridians/parallels 
ax2.coastlines(linewidth=0.75, color='black', resolution='50m')
ax2.gridlines(crs=ccrs.PlateCarree(), linestyle='-')
ax2.set_extent([-180, 180, 50, 90], crs=ccrs.PlateCarree())



# Model-Obs
ax3 = plt.subplot(3, 1, 3, projection=ccrs.NorthPolarStereo(central_longitude=0))
(c_sic - da_51.sel(time=cdate)).plot.pcolormesh(ax=ax3, x='lon', y='lat', 
                                     transform=ccrs.PlateCarree(),
                                     cmap=cmap_dif,
                      vmin=-1, vmax=1)
ax3.set_title('GFDL - NSIDC 0051')
# Add coastlines and meridians/parallels 
ax3.coastlines(linewidth=0.75, color='black', resolution='50m')
ax3.gridlines(crs=ccrs.PlateCarree(), linestyle='-')
ax3.set_extent([-180, 180, 50, 90], crs=ccrs.PlateCarree())


plt.tight_layout()